In [2]:
# Load the python modules.
import os, sys
#Insert the common folder in the path (to be able to load python module)
sys.path.insert(0,os.path.join(os.path.pardir,'common'))
import numpy as np
import wget
from dapper import Chronology

from toolbox import load_config, path, get_filenames, load_data,rmse_norm
from l2s_utils import plot_L96_2D, default_param

#To control plot layouts
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("paper")

from tqdm.notebook import tqdm

# Setting

## Source of the data
The variable ```source``` can be set to ```"ftp"``` or ```"local"```.
- ```ftp```: it will download the data from an ftp server, so there is no need to compute the data. WARNING: it will override existing data in the data simulations data directory.
- ```local```: it will load the data locally. So you have to first either having downloaded the data from the ftp server, either having computing the data using the other notebooks.

## Configuration files
There are 4 configuration file to set. Default configuration files are provided but if you want to make further experiment, it is possible to change the configuration file.
- ```config_path```: contains the path of the data directory.
- ```config_ftp```: contains the url of the data in the ftp server. This configuration file is optional if you don't download the data from ftp.
- ```config_ref```: contains the reference simulations settings.
- ```config_sens```: contains the different settings for an sensitivity experiment.

In [3]:
#Source of the data (comment/uncomment the lines given your choice)
#source = 'local'
source = 'ftp'


In [4]:
# Directory where to save the figure (if figdir = None, no figure are saved)
# figdir = '.'
figdir = None

In [5]:
#Configuration files
config_path = 'config/paths.yml'
config_ftp = 'config/ftp.yml' #optional if data are generated locally
config_ref = 'config/ref_test.yml'
config_sens = 'config/sens_test.yml'
config_sens_po = 'config/sens_test_po.yml'

In [6]:
# Load the paths (yml file)
paths = load_config(config_path)

# Load the reference
ref = load_config(config_ref)

# Rootdir for the L2S data
rootdir = path(paths['rootdir'])

# Template names
template_ref = ref['template']

#Check if figdir exist and set the figdir accordingly
savefig = bool(figdir)

#Directory containing the simulations
refdir = path(os.path.join(rootdir,ref['savedir']))

#Print:
print("Directory containing the simulations for the reference experiment:\n->",refdir)


Directory containing the simulations for the reference experiment:
-> /Users/brajard/Documents/recherche/papers/2020/PHILTRA/reddaml/data/L2S/simul/test


In [7]:
#Load the ftp configuration file if needed
if source == 'ftp':
    ftp = load_config(config_ftp)
    ftpurl, ftpdir = ftp['url'], ftp['test']
    print('\033[93m'+'Warning! The source is set to ftp. \nIt will override existing local simulation in the simulation directory'+'\033[0m')
else:
    ftp = None
    ftpurl, ftpdir = None, None

Warning! The source is set to ftp. 
It will override existing local simulation in the simulation directory


## Plot Figure 2

In [8]:
sens_da = load_config(config_sens)
sens_po = load_config(config_sens_po)

# Define the parameters
used_parameter = { 'p', 'std_o', 'dtObs', 'std_m' ,'N','T','seed', 'Nfil_train'}
lparam_da = {k:sens_da.get(k,[default_param[k]]) for k in used_parameter}
lparam_po = {k:sens_po.get(k,[default_param[k]]) for k in used_parameter}
print('Parameters of the sensitivity experiment:\n',lparam_da)
print('Parameters of the sensitivity experiment (perfect observations):\n',lparam_po)

# Check if the the parameter dtObs (should be the same in the sensitivity experiments with perfect obs)
assert lparam_da['dtObs'] == lparam_po['dtObs']

#load ref file
ref = load_config(config_ref)

ref_params = {k:ref.get(k,[default_param[k]]) for k in used_parameter}

sensdir = path(os.path.join(rootdir,sens_da['savedir']))

Parameters of the sensitivity experiment:
 {'Nfil_train': [1, 5], 'std_m': [0.06], 'T': [15.0], 'N': [20], 'std_o': [0.05, 0.1, 0.2, 0.5, 1.0, 2.0], 'p': [18, 27, 36], 'dtObs': [0.01, 0.05, 0.1, 0.15], 'seed': [10]}
Parameters of the sensitivity experiment (perfect observations):
 {'Nfil_train': [1], 'std_m': [0.06], 'T': [15.0], 'N': [20], 'std_o': [1], 'p': [36], 'dtObs': [0.01, 0.05, 0.1, 0.15], 'seed': [10]}


In [9]:
# List all the experiment with the corresponding filenames
templates_da = {k:v for k,v in sens_da['template'].items() if k in {'truth','trunc','hybrid'}} 
templates_po = {k:v for k,v in sens_po['template'].items() if k in {'hybrid'}} 
dsens_da = get_filenames(lparam_da, templates_da)
dsens_po = get_filenames(lparam_po, templates_po)

# Change the name of the perfect hybrid model to not overide the key
dsens_po['perfect'] = dsens_po.pop('hybrid')
dsens = {**dsens_da,**dsens_po}
print('Types of run for the experiment:',dsens.keys())


Types of run for the experiment: dict_keys(['truth', 'trunc', 'hybrid', 'perfect'])


In [10]:
# Load trunc/true

#Set the filename (as a str) of the data
fname = str(dsens['truth'].name.iloc[0])
data_truth = load_data(sensdir,fname,ftpurl=ftpurl,ftpdir=ftpdir)

print("Data file truth:\n->",fname)

#Set the filename (as a str) of the data
fname = str(dsens['trunc'].name.iloc[0])
data_trunc = load_data(sensdir,fname,ftpurl=ftpurl,ftpdir=ftpdir)

print("Data file trunc:\n->",fname)


ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-truth-T15.0-N20-seed10.npz
Data file truth:
-> simul-truth-T15.0-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-trunc-T15.0-N20-seed10.npz
Data file trunc:
-> simul-trunc-T15.0-N20-seed10.npz


In [11]:
# Format the data (the time step of the trunc model is different from the time step of the real model)


#Trunc data
xx_trunc = data_trunc['xx']

#chronology
chrono_trunc = Chronology(dkObs=1,T=data_trunc['T'],dt=data_trunc['dt'])
chrono_truth = Chronology(dkObs=1,T=data_truth['T'],dt=data_truth['dt'])

#Compute the time step subsample:
dk = int(chrono_trunc.dt/chrono_truth.dt)

#Compute the size of the trunc model stat:
nU = xx_trunc.shape[-1]

#Subsampe the true model in space and time to fit the trunc model space
xx_truth = data_truth['xx'][::dk,:,:nU]

#Define the time axis
tt = chrono_trunc.tt

#Display the size of the model simulations
print('Size of the time axis:',tt.shape)
print('Size of the truth simulation (time step, members, space):',xx_truth.shape)
print('Size of the trunc simulation (time step, members, space):',xx_trunc.shape)

Size of the time axis: (1501,)
Size of the truth simulation (time step, members, space): (1501, 20, 36)
Size of the trunc simulation (time step, members, space): (1501, 20, 36)


In [12]:
#Reformat DataFrame
import pandas as pd

#Add a column in the Data frame with the type of simulation
for k in dsens:
    dsens[k]=dsens[k].assign(model=k)

#Concatenate all the dataframe in one:
df = pd.concat([v for k,v in dsens.items()], ignore_index=True)
df

,N,Nfil_train,T,dtObs,p,seed,std_m,std_o,name,model
0,20,1,15.0,0.01,18,10,0.06,0.05,simul-truth-T15.0-N20-seed10.npz,truth
1,20,1,15.0,0.01,18,10,0.06,0.05,simul-trunc-T15.0-N20-seed10.npz,trunc
2,20,1,15.0,0.01,18,10,0.06,0.05,simul-hybrid-p18-dtObs0.01-std_o0.05-filter1-N...,hybrid
3,20,1,15.0,0.01,18,10,0.06,0.10,simul-hybrid-p18-dtObs0.01-std_o0.1-filter1-N2...,hybrid
4,20,1,15.0,0.01,18,10,0.06,0.20,simul-hybrid-p18-dtObs0.01-std_o0.2-filter1-N2...,hybrid
...,...,...,...,...,...,...,...,...,...,...
145,20,5,15.0,0.15,36,10,0.06,2.00,simul-hybrid-p36-dtObs0.15-std_o2.0-filter5-N2...,hybrid
146,20,1,15.0,0.01,36,10,0.06,1.00,simul-perfect-dtObs0.01-N20-seed10.npz,perfect
147,20,1,15.0,0.05,36,10,0.06,1.00,simul-perfect-dtObs0.05-N20-seed10.npz,perfect
148,20,1,15.0,0.10,36,10,0.06,1.00,simul-perfect-dtObs0.1-N20-seed10.npz,perfect


In [ ]:
#Compute the RMSE os all the  simulations
# If ftp is True, it can take some few minutes
RMSE = dict()

#Time to compute the RMSE
it = 200

df.assign(RMSE = np.nan) #Default value (changed in the loop)
print('->compute RMSE for the all the models (except the truth)')
for idx in tqdm(df[df.model != 'truth'].index):
    fname = df.loc[idx,'name']
    data = load_data(sensdir,fname,ftpurl=ftpurl,ftpdir=ftpdir)
    xx_est = data['xx']
    RMSE[idx] = rmse_norm(xx_truth,xx_est)
    df.loc[idx,'RMSE'] = RMSE[idx][it]
    data.close()


->compute RMSE for the all the models (except the truth)


ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-trunc-T15.0-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p18-dtObs0.01-std_o0.05-filter1-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p18-dtObs0.01-std_o0.1-filter1-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p18-dtObs0.01-std_o0.2-filter1-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p18-dtObs0.01-std_o0.5-filter1-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p18-dtObs0.01-std_o1.0-filter1-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p18-dtObs0.01-std_o2.0-filter1-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p27-dtObs0.01-std_o0.05-filter1-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p27-dtObs0.01-std_o0.1-filter1-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p27-dtObs0.01-std_o0.2-filter1-N20-seed10.npz
ftp://ftp.ners

ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p27-dtObs0.01-std_o0.2-filter5-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p27-dtObs0.01-std_o0.5-filter5-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p27-dtObs0.01-std_o1.0-filter5-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p27-dtObs0.01-std_o2.0-filter5-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p36-dtObs0.01-std_o0.05-filter5-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p36-dtObs0.01-std_o0.1-filter5-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p36-dtObs0.01-std_o0.2-filter5-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p36-dtObs0.01-std_o0.5-filter5-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p36-dtObs0.01-std_o1.0-filter5-N20-seed10.npz
ftp://ftp.nersc.no/reddaml/L2S/simul/test/simul-hybrid-p36-dtObs0.01-std_o2.0-filter5-N20-

In [ ]:
#Initialise the figure
import matplotlib.pyplot as plt
fig, (ax, ax0,ax1) = plt.subplots(ncols=3,figsize=(8.1,3),sharey='all')


#Plot the sensitivity to dtobs
df_dthyb = df[(df['Nfil_train']==ref_params['Nfil_train'][0])&
          (df['std_o']==ref_params['std_o'][0])&
          (df['p']==ref_params['p'][0]) & (df['model']=='hybrid')
         ]

df_dtpo = df[df['model']=='perfect']
df_trunc = df[df['model']=='trunc']

x = np.arange(df_dthyb.shape[0])
xtrunc = [len(x)]
ax0.bar(x=x-0.2,height=df_dthyb['RMSE'],width=0.4,color='C1',label='DA hyb.');
ax0.bar(x=x+0.2,height=df_dtpo['RMSE'],width=0.4,color='C2',label='perf.obs. hyb.');
ax0.bar(x=xtrunc,height=df_trunc['RMSE'],width=0.4,color='C3',label='trunc.')
ax0.set_xticks(list(x)+xtrunc);
ax0.set_xticklabels([str(dt) for dt in df_dthyb['dtObs']]+['trunc.']);
ax0.bar(x=x[1]-0.2,height=df_dthyb.iloc[1]['RMSE'],width=0.4,fill=False,LineWidth=2,edgecolor='black',label='ref.exp.')

ax0.legend()
ax0.set_xlabel('$\Delta t$')

#Plot the sensitivity to the filtration
df_filhyb = df_stdo = df[(df['dtObs']==ref_params['dtObs'][0]) & 
          (df['p']==ref_params['p'][0]) &
          (df['Nfil_train'].isin([1,5])) & (df['model']=='hybrid')
         ]
df_1 = df_filhyb[df_filhyb['Nfil_train']==1]
df_5 = df_filhyb[df_filhyb['Nfil_train']==5]


x = np.arange(df_1.shape[0])
ax1.bar(x=x-0.2,height=df_1['RMSE'],
        width=0.4,color='C1',label='no filter');
ax1.bar(x=x+0.2,height=df_5['RMSE'],
        width=0.4,color='C4',label='filter 0.05 MTU');
ax1.bar(x=x[1]-0.2,height=df_1.iloc[1]['RMSE'],
        width=0.4,fill=False,LineWidth=2,edgecolor='black',label='ref.exp.')
ax1.set_xticks(x);
ax1.set_xticklabels(['{}'.format(stdo) for stdo in df_1['std_o']]);
ax1.set_xlabel('$\sigma^o$')
ax1.legend()

#Plot the sensitivity to the density of observations (time series)
#Plot the sensitivity to dtobs

#Number time step to display
limT = 800

#Displayed color
cols=['C5','C6','C1']

df_phyb = df[(df['Nfil_train']==ref_params['Nfil_train'][0])&
          (df['std_o']==ref_params['std_o'][0])&
          (df['dtObs']==ref_params['dtObs'][0]) & 
             (df['model']=='hybrid')
         ]

df_ppo = df[(df['model']=='perfect') & 
            (df['dtObs']==ref_params['dtObs'][0])]

for i,idx in enumerate(df_phyb.index):
    ax.plot(tt[:limT],RMSE[idx][:limT],
            label='$N_y$={}'.format(df_phyb.loc[idx,'p']),
           color=cols[i])
ax.plot(tt[:limT],RMSE[df_trunc.index[0]][:limT],
        label='trunc.',color='C3',linewidth=2)

ax.plot(tt[:limT],RMSE[df_ppo.index[0]][:limT],
        label='perf. obs.',color='C2',linewidth=2)

ax.legend();

if figdir:
    fig.savefig(os.path.join(figdir,'philtra-fig2.png'),
            dpi=200, bbox_inches='tight', pad_inches=0)
